<a href="https://colab.research.google.com/github/DRWang3/MPM_testing/blob/main/skeleton_drw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
#@title Model Driver Class
#from IPython.display import display, HTML
#display(HTML('<h1 style="color: blue; background-color: lightyellow; padding: 10px; border-radius: 5px;">Custom Header</h1>'))

import numpy as np
import pandas as pd

class model_driver:
    """Driver class for taking in input files and running the simulation.

    Attributes:
        drivers_filename (str): Path to the drivers CSV file.
        parameter_filename (str): Path to the parameters CSV file.
    """

    def __init__(self, drivers_filename, parameter_filename):
        self.drivers_filename = drivers_filename
        self.parameter_filename = parameter_filename

    def read_input_files(self):
        # Read input CSV files for drivers and parameters
        drivers = pd.read_csv(self.drivers_filename)
        params = pd.read_csv(self.parameter_filename, header=None)
        params_dict = params.set_index(0).to_dict()[1]
        return drivers, params_dict

    def run_simulation(self):
        drivers, params_dict = self.read_input_files()

        for index, row in drivers.iterrows():
            DOY = row['DOY']
            latitude = row['latitude']

            # atmosphere object for computing atmospheric properties at this timestep
            atmosphere_instance = atmosphere(DOY, latitude)
            atmospheric_properties = atmosphere_instance.get_atmospheric_properties()

            exogenous_inputs = {
                'Solar_Constant': atmospheric_properties['Solar_Constant'],
                'Sin_Solar_Declination': atmospheric_properties['Sin_Solar_Declination'],
                'Cos_Solar_Declination': atmospheric_properties['Cos_Solar_Declination'],
                'Day_Length': atmospheric_properties['Day_Length'],
                'Photoperiod_Day_Length': atmospheric_properties['Photoperiod_Day_Length'],
                'Daily_Sin_Beam_Exposure': atmospheric_properties['Daily_Sin_Beam_Exposure'],
                'temperature': row['temperature'],
                'radiation': row['radiation'],
                'precipitation': row['precipitation'],
                'wind_speed': row['wind_speed'],
                'VPD': row['VPD'],
                'latitude': row['latitude']
            }

            # environment object that acts as interface between plant and non-plant




            # testing
            # print(exogenous_inputs)



In [58]:
#@title Atmosphere Class
class atmosphere:
    def __init__(self, DOY, latitude):
        self.rad = np.pi / 180
        self.lat = latitude
        self.doy = DOY

    def get_atmospheric_properties(self):
        """
        This function calculates atmospheric properties for a model timestep
        Args:
            DOY (int): Day of the year (1-365).
        """

        Sun_Angle_Inclination = -2
        dec = np.arcsin(np.sin(23.45 * self.rad) * np.cos(2 * np.pi * (self.doy + 10) / 365))
        Sin_Solar_Declination = np.sin(self.rad * self.lat) * np.sin(dec)
        Cos_Solar_Declination = np.cos(self.rad * self.lat) * np.cos(dec)
        angle_factor = Sin_Solar_Declination / Cos_Solar_Declination

        Day_Length = 12.0 * (1 + 2 * np.arcsin(angle_factor) / np.pi)
        Photoperiod_Day_Length = 12.0 * (1 + 2 * np.arcsin((-np.sin(Sun_Angle_Inclination * self.rad) + Sin_Solar_Declination) / Cos_Solar_Declination) / np.pi)
        Daily_Sin_Beam_Exposure = 3600 * (Day_Length * (Sin_Solar_Declination + 0.4 * (Sin_Solar_Declination**2 + Cos_Solar_Declination**2 * 0.5)) +
                             12.0 * Cos_Solar_Declination * (2.0 + 3.0 * 0.4 * Sin_Solar_Declination) * np.sqrt(1.0 - angle_factor**2) / np.pi)
        Solar_Constant = 1367 * (1 + 0.033 * np.cos(2 * np.pi * (self.doy - 10) / 365))

        return {
            'Solar_Constant': Solar_Constant,
            'Sin_Solar_Declination': Sin_Solar_Declination,
            'Cos_Solar_Declination': Cos_Solar_Declination,
            'Day_Length': Day_Length,
            'Photoperiod_Day_Length': Photoperiod_Day_Length,
            'Daily_Sin_Beam_Exposure': Daily_Sin_Beam_Exposure
        }


In [61]:
#@ Example Usage

drivers = 'https://raw.githubusercontent.com/DRWang3/MPM_testing/refs/heads/main/generated_hourly_weather_data.csv'
params =  'https://raw.githubusercontent.com/DRWang3/MPM_testing/refs/heads/main/parameters.csv'

driver = model_driver(drivers, params)
driver.run_simulation()